In [1]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.preprocessing import LabelEncoder
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import math
from tqdm.notebook import tqdm
import sys
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download("stopwords")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nabr9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
cd drive/MyDrive/datasets

[Errno 2] No such file or directory: 'drive/MyDrive/datasets'
/content


## 1. Представление и предобработка текстовых данных в виде последовательностей

1.1 Представьте первое предложение из строки `text` как последовательность из индексов слов, входящих в это предложение

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [ ]:
text_list = word_tokenize(re.sub('[^a-zA-z0-9]', ' ', sent_tokenize(text)[0])); [text_list.index(i) for i in text_list]

[0, 1, 2, 3, 4, 5, 6, 7]

1.2 Представьте первое предложение из строки `text` как последовательность векторов, соответствующих индексам слов. Для представления индекса в виде вектора используйте унитарное кодирование. В результате должен получиться двумерный тензор размера `количество слов в предложении` x `количество уникальных слов`

In [ ]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

In [ ]:
onehot = torch.zeros([len(text_list), len(np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text))))]); onehot

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]])

In [ ]:
uniword = np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text))).tolist()

In [ ]:
for i in range(len(text_list)):
  if text_list[i] in uniword:
    indexx = uniword.index(text_list[i])
    onehot[i][indexx] = 1
print(onehot)

tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.]])


1.3 Решите задачу 1.2, используя модуль `nn.Embedding`

In [ ]:
embedding = nn.Embedding(len(np.unique(word_tokenize(re.sub('[^a-zA-z0-9]', ' ', text)))), len(text_list))

Parameter containing:
tensor([[-1.5796,  1.1445, -0.5662,  1.1377,  1.2632, -0.3197, -0.1794, -0.3176],
        [-0.3157, -0.0390,  2.0914,  1.8927, -1.7984, -0.7977,  0.8658,  0.0178],
        [-1.1207,  0.0529, -0.6721, -1.3481, -1.2635,  0.8415,  0.5953,  0.7191],
        [ 1.7196,  0.7924, -2.3789,  0.6223,  2.1142,  0.4513,  2.4166, -0.0963],
        [ 0.0892,  1.4384,  1.6088,  0.4611,  1.4338,  0.0793,  0.1988, -0.5858],
        [ 0.5116, -0.2178,  0.1307,  0.3669, -0.9371,  0.0352, -0.1094, -0.3817],
        [ 1.9997, -1.0942, -0.1997, -0.2700, -0.9381,  2.1229,  1.4419,  0.3844],
        [-1.2568,  1.4662,  0.2568, -0.1948,  0.1050,  0.9160, -0.0316, -0.6167],
        [ 1.0667,  0.1981,  0.8875, -0.4200,  0.1048,  0.2092, -0.5582, -0.0816],
        [ 0.2487, -0.8823, -0.1489, -0.0980, -0.8303,  0.8666,  0.2572,  0.2306],
        [-0.8850, -0.2181, -0.0355, -1.0359,  0.5875,  1.2025,  1.0370,  0.6809],
        [-1.2965,  0.7015,  0.2445,  0.0564,  1.4246, -1.2176,  0.7540, -0.5

In [ ]:
for_indexes = torch.zeros(8)
for i in range(len(text_list)):
  if text_list[i] in uniword:
    for_indexes[i] = uniword.index(text_list[i])

In [ ]:
for_indexes.long()

tensor([ 4, 24, 16,  6, 18, 22, 11,  8])

In [ ]:
embedding(for_indexes.long())

tensor([[ 0.9685, -1.7910,  0.2274, -0.4698,  0.1916, -0.7824, -0.3267, -1.4738],
        [-0.4715, -1.0201,  0.5779,  0.0176,  0.6359, -0.2606, -1.2780, -0.2258],
        [-0.2825,  0.6390,  0.8526,  0.2445,  1.7588, -0.4913, -0.7699,  0.6086],
        [-0.4460, -0.5661,  0.7954, -0.0893, -1.9659, -0.5617, -1.1380,  0.2429],
        [ 1.4158, -1.0546, -0.6608,  0.8766, -1.0432,  1.3776, -2.8067, -0.0651],
        [ 0.8337, -1.5029,  1.8954, -0.8557, -0.6300,  0.2706, -1.2130,  0.4508],
        [ 0.4607,  0.3549, -0.1438,  1.3835, -0.9604,  1.5546, -0.3867,  0.3384],
        [-0.7094, -0.1978,  0.1911, -0.6461, -1.0215,  0.9643, -1.7661, -1.4789]],
       grad_fn=<EmbeddingBackward0>)

## 2. Классификация фамилий по национальности (ConvNet)

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`. 

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)
  * добавьте в словарь специальный токен `<PAD>` с индексом 0
  * при создании словаря сохраните длину самой длинной последовательности из набора данных в виде атрибута `max_seq_len`

2.5 Реализовать класс `SurnamesDataset`
  * метод `__getitem__` возвращает пару: <последовательность индексов токенов (см. 1.1 ), номер класса> 
  * длина каждой такой последовательности должна быть одинаковой и равной `vocab.max_seq_len`. Чтобы добиться этого, дополните последовательность справа индексом токена `<PAD>` до нужной длины

2.6. Обучить классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding`. Рассмотрите два варианта: 
    - когда токен представляется в виде унитарного вектора и модуль `nn.Embedding` не обучается
    - когда токен представляется в виде вектора небольшой размерности (меньше, чем размер словаря) и модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

In [30]:
data = pd.read_csv('surnames.csv')
data['nationality'] = pd.factorize(data['nationality'])[0]
result_df = data.drop_duplicates(subset=['surname'], keep='first')
y = result_df['nationality']
X = result_df['surname']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
class Vocab:
  def __init__(self, data):
        letters = ['<PAD>']
        for i in data:
            for j in i:
                if j not in letters:
                    letters.append(j)
        self.idx_to_token = {idx: val for idx, val in enumerate(letters)}
        self.token_to_idx = {val: idx for idx, val in self.idx_to_token.items()}
        self.vocab_len = len(self.idx_to_token)
        self.max_seq_len = max([len(i) for i in data])

In [32]:
# vocab = Vocab(X)

In [33]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab
    
    def vectorize(self, surname):
        abc = []
        for i in surname.lower():
            if i in list(self.vocab.idx_to_token.values()):
                get_index = list(self.vocab.idx_to_token.values()).index(i)
                abc.append(get_index)
        while len(abc)<self.vocab.max_seq_len:
            abc.append(0)
        return torch.tensor(abc, dtype=torch.int64)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return self.vectorize(self.X.iloc[index]), self.y.iloc[index]

In [34]:
# class Vocab:
#   def __init__(self, data: list):
#     unique_ch = ["<PAD>"] + list(set("".join(data)))
#     ch_ind = list(np.arange(len(unique_ch)))
#     self.idx_to_token = dict(zip(ch_ind, unique_ch))
#     self.token_to_idx = dict(zip(unique_ch, ch_ind))
#     self.vocab_len = len(unique_ch)
#     self.max_seq_len = max(list(map(len, data)))


# class SurnamesDataset(Dataset):
#   def __init__(self, X, y, vocab: Vocab):
#     self.X = X
#     self.y = y
#     self.vocab = vocab

#   def get_idx(self, surname):
#     idx = [0] * self.vocab.max_seq_len
#     for i, c in enumerate(surname):
#       idx[i] = self.vocab.token_to_idx[c]
    
#     return torch.tensor(idx, dtype=torch.int64)


#   def __len__(self):
#     return len(self.X)

#   def __getitem__(self, idx):
#     return self.get_idx(self.X.iloc[idx]), self.y.iloc[idx]

In [35]:
list(surname_train)

NameError: name 'surname_train' is not defined

In [36]:
vocab = Vocab(X.tolist())
surname_train = SurnamesDataset(X_train, y_train, vocab)
surname_test = SurnamesDataset(X_test, y_test, vocab)

In [37]:
test_ds = SurnamesDataset(X_test, y_test, Vocab(X))
train_ds = SurnamesDataset(X_train, y_train, Vocab(X))

In [38]:
batch_size = 64
train_iter = DataLoader(train_ds, batch_size, shuffle=True)
test_iter = DataLoader(test_ds, batch_size, shuffle=False)

In [39]:
# class fam_model(nn.Module):
#     def __init__(self, input_size, vocab_size, num_channels, output_size):
#         super(fam_model, self).__init__()
#         self.input_size = input_size
#         self.vocab_size = vocab_size
#         self.output_size = output_size
#         self.embedding = nn.Embedding(self.vocab_size, input_size)
#         self.lr1 = nn.Conv1d(in_channels=input_size, out_channels=num_channels, kernel_size=3)
#         self.lr2 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
#         self.lr3 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
#         self.lr4 = nn.Conv1d(in_channels=num_channels, out_channels=num_channels, kernel_size=3)
#         self.relu = nn.ELU()
#         self.norm = nn.BatchNorm1d(num_channels)
#         self.maxpool = nn.MaxPool1d(kernel_size = 3)
#         self.lr5 = nn.Linear(num_channels, self.output_size)
        
#     def forward(self, x):
#         out = self.embedding(x)
#         out = self.lr1(out)
#         out = self.norm(out)
#         out = self.relu(out)
#         out = self.lr2(out)
#         out = self.norm(out)
#         out = self.relu(out).squeeze(dim=2)
#         out = self.lr5(out)
#         return torch.nn.functional.log_softmax(out, dim=1)

In [40]:
## Модель 1

class sur_model(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size):
        super(sur_model, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.seqnet = nn.Sequential(nn.Embedding(self.vocab_size, input_size),
            nn.Conv1d(input_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(512, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
      out = self.seqnet(x)
      out = self.fc(out)
      return torch.nn.functional.log_softmax(out, dim=1)

In [41]:
input_size = vocab.max_seq_len
vocab_size = vocab.vocab_len
output_size = len(set(data['nationality']))
num_channels = 300
model = sur_model(input_size, vocab_size, num_channels, output_size)

In [42]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [51]:
def train(model, device, train_dl, optimizer, total):
    #set model in train() mode:
    model.train()
    
    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0
    
    for i, (inputs, targets) in tqdm(enumerate(train_dl), total= total, desc='Training minibatch loop '):
        # Forward pass
        outputs = model(inputs)
        print(targets)
        loss = criterion(outputs, targets)
        
        # Backward and optimize        
        # zero grad before new step        
        optimizer.zero_grad()                        
        loss.backward()
        optimizer.step()   

        # calculating the total_loss for checking
        total_loss += loss           
        
        # PREDICTIONS 
        total_samples += targets.shape[0]   
        _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
        correct_samples += torch.sum(predictions_indices==targets)

    train_accuracy = float(correct_samples) / total_samples        
    
    return total_loss, train_accuracy                        

In [52]:
# TESTING THE MODEL
def test(model, device, test_dl, total):
    #set model in eval() mode (it skips Dropout etc):
    model.eval()
    
    total_samples = 0.0
    correct_samples = 0.0 
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(test_dl), total=total, desc='Testing minibatch loop:'):                      
            # apply model to input data
            outputs = model(inputs)        
                       
            #PREDICTIONS
            total_samples += targets.shape[0]   
            _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
            correct_samples += torch.sum(predictions_indices==targets)                    
        
    test_accuracy = correct_samples / total_samples        
    
    return test_accuracy              

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [54]:
%%time
for_losses = []
n_total_steps = len(train_iter)


for epoch in range(25):
    total_loss, train_accuracy = train(model, device, train_iter, optimizer, total = math.ceil(len(train_ds)/64))
    test_accuracy = test(model, device, test_iter, total = math.ceil(len(test_ds)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

tensor([ 3,  4,  9,  7,  3,  0,  3,  0,  0,  2,  0,  0,  3,  4,  6,  3,  0,  0,
         0,  5,  4,  0,  1,  6,  0, 11,  5,  0,  0,  3,  3,  0,  9,  7, 13,  4,
         3, 11,  3,  3,  0,  5,  0,  3,  9,  0,  0,  3,  4, 17,  9, 17, 12,  0,
        14,  3,  0,  0,  6,  0,  0,  3,  0,  6])
tensor([ 4,  3,  0,  0,  3,  0,  6,  0,  3,  0,  0,  0,  3,  0,  3,  3,  3,  2,
         0,  0,  7,  0,  8,  3,  3,  0,  3,  6,  3, 11,  0,  4,  1,  0,  0,  4,
         1,  0,  4,  1,  0,  6,  8,  0,  6,  0,  9,  9,  3,  0, 11,  0,  1,  3,
         3,  0,  4, 12,  4,  5,  2,  3,  0,  3])
tensor([ 3,  3,  3,  3,  6,  0,  4,  9,  0,  3,  3,  0,  0,  3, 13,  0,  4,  0,
         4,  0,  6,  7,  0,  3,  7,  0,  0,  9,  9,  4,  0, 11,  6,  6,  1,  3,
         0, 13,  0,  8,  3,  6,  3,  4,  3,  0,  3,  3,  0,  0,  0,  0,  3,  3,
         3,  0,  0,  3,  0,  1,  3,  0,  3, 13])
tensor([13,  0,  0,  0,  3, 13,  0,  9,  0,  4,  0,  7,  6,  0,  7,  0,  2,  0,
        17,  7,  3,  0,  3, 11,  0,  0,  0,  1,  0,  

tensor([10,  3,  1,  3,  0,  7,  0,  3,  2,  0,  3,  0,  6,  3, 13,  4,  0,  4,
         3,  0,  0,  0,  4,  1,  3,  3,  0,  0,  4,  8,  0,  4,  0,  3, 13,  9,
         0,  4,  4,  4,  0, 12,  0,  6,  3, 15,  0,  0,  0,  0,  7,  6,  3, 13,
         0, 10,  6,  6,  0,  0,  9,  9, 13,  0])
tensor([ 0, 11,  0,  0,  3, 15,  1, 13,  0,  4,  3,  0,  9,  0,  7, 13,  3,  0,
         3,  9,  0,  0,  0,  3, 13,  0,  1,  3,  7,  3,  6,  0,  0,  0, 11,  0,
         0,  3,  3,  5, 10,  3,  3,  3,  0,  0,  6,  0, 13,  6,  7,  3,  7,  3,
         0,  3, 12,  6,  0,  3,  8,  4,  0,  0])
tensor([ 7,  0,  5,  0,  0,  0,  4,  3,  0,  6,  3,  3, 11,  0,  3,  4,  9,  3,
         3,  4,  4,  0,  0,  0,  3,  3,  3,  3,  3,  3,  3,  3, 10,  0,  3,  3,
         9,  0,  7,  0,  0,  3,  0,  0,  0,  0,  0,  0,  3,  3, 17,  9,  5,  3,
        11,  9,  0,  3,  1,  0,  0,  0,  6,  3])
tensor([ 0,  1,  3,  3,  3,  3,  3,  0, 17,  0, 13,  0,  0,  4,  9, 15,  0,  3,
         3,  0,  0,  0,  6,  4,  0,  4,  0,  4,  0,  

tensor([ 0,  0,  3, 17,  3,  0,  0, 11,  8,  0,  6,  7,  3,  0,  6,  0,  3,  3,
         9,  0,  8,  3,  0,  6,  0,  0,  6,  0, 13,  0,  6,  4,  6,  9,  0,  9,
         3,  7,  3,  5, 16,  0,  9,  3,  4,  4,  6,  0, 11,  0,  0,  9,  3,  3,
         3,  0,  0,  3,  0,  0,  9,  0,  7,  3])
tensor([ 3,  6,  6,  0,  4,  3,  0,  0,  0,  3,  6,  3,  3,  0,  0,  0,  3,  6,
         6,  0,  4,  3,  4,  7, 16,  6,  9,  3,  0,  0,  4,  0,  0,  0,  3,  0,
         3,  3,  3,  3,  3,  3,  0, 17,  0,  8,  3,  9,  3,  7,  8,  0,  9,  7,
         7,  6,  0,  3,  3,  2,  0,  3,  3, 13])
tensor([ 6,  6,  5,  0,  0,  3,  0,  0,  3,  0,  9,  3,  3,  3,  8,  4,  3,  4,
         4,  0,  9,  4,  0,  0,  0, 13,  0,  3,  4,  3,  0, 11,  0,  3,  0,  0,
        13,  3,  4,  3,  0,  9,  3, 17,  7,  8,  3,  4,  5,  0,  3, 11,  3,  0,
         3,  3,  4,  7, 13,  3, 10,  3,  0,  3])
tensor([ 3,  3,  3,  3,  3,  3,  4,  6,  0,  0,  0,  0,  6,  3,  3,  0,  0,  5,
         4,  0,  6,  0,  3,  6,  7,  4, 11,  9,  0,  

tensor([ 3,  3,  9,  5,  3, 12,  4,  3,  0,  3,  0, 10, 13,  0,  6,  3,  4, 16,
         3, 15,  3,  9,  3,  3,  9,  4,  9,  0,  4,  0,  5, 13,  0,  3,  4,  6,
         3, 10,  0,  4,  7,  2,  3,  4,  3,  6,  0, 10,  0,  1,  3, 11,  3,  6,
         0,  0,  0,  0,  0,  9,  8,  3,  7,  0])
tensor([ 3,  0,  0, 15,  0,  4,  4,  6,  3,  2,  3,  9,  0,  3,  0,  6,  6,  3,
         0,  3,  3,  3,  3,  9,  4,  0,  0,  0,  3,  1,  0,  3,  0,  9,  0,  3,
         3,  9,  0,  5, 12,  3, 10,  9, 13,  0,  0,  0,  6,  3,  4,  0,  3,  0,
        11,  0,  3,  4,  0,  0,  3,  3,  0,  0])
tensor([ 9,  2,  4,  3,  5,  3, 17, 17,  0,  3,  8,  0, 14,  3,  5,  9,  9,  0,
         1,  0, 13,  3,  0,  7, 16,  3,  0,  0,  9, 13,  1,  1,  3,  3,  4, 14,
        13,  3,  0,  0,  3,  1,  0,  3,  0, 11,  7,  8,  0,  3, 11,  3, 16,  1,
        16,  4, 13,  3,  0,  9,  7,  8,  2,  4])
tensor([ 3,  9,  0,  4,  0,  3,  9,  0,  0,  3,  0, 11,  0,  6,  0,  6, 10,  3,
        16,  3,  7,  3,  3,  9, 16, 15, 10,  3,  3,  

...

tensor([12,  1,  0,  5,  5,  9,  0,  3,  3,  3,  0,  3,  6,  4,  9,  3,  0,  3,
         7, 12,  3,  3,  3,  3,  0,  3,  3, 11,  0,  3,  0,  3,  6,  0, 10,  9,
         4,  3,  8,  6,  3,  0,  0,  3,  3,  0,  3,  3,  9,  3,  7,  3,  8,  1,
        13,  0,  1,  0,  3,  0,  3,  5,  4,  0])
tensor([ 2,  0,  0,  3,  3,  4,  2,  0,  3,  6,  4,  0,  0,  0, 11,  3,  0,  3,
         3,  3,  3, 11,  5,  7,  0,  9,  4,  0,  8,  3,  7,  8, 10,  3,  0,  0,
         9,  3,  0,  6,  4,  0,  3,  0,  9,  3,  0,  6,  5,  3, 12,  0,  3,  3,
         2,  3,  4,  3,  0, 12,  0,  3,  0,  3])
tensor([ 0, 10,  0,  1,  6, 15,  4,  9,  5,  3,  3, 10,  6, 11,  0,  0,  0,  0,
         0,  2,  0,  9,  3,  0,  3,  3,  9,  3,  3,  3, 11,  9,  3,  3,  3,  4,
         0,  9,  0,  3,  3,  0,  0,  0,  3,  4,  0,  7,  3,  0,  0,  0,  3,  3,
         0,  3,  0,  3,  6,  3,  3,  0,  3,  0])
tensor([ 4,  0,  7,  0,  6,  9,  0,  9,  0, 10,  0, 13,  3,  0,  9,  4,  7, 11,
         4,  3,  3,  6,  3,  3,  7,  0,  3,  3,  0,  

tensor([ 3,  0,  4,  3,  3,  4,  6,  3, 10, 17,  7,  0,  9,  7,  3,  3,  6,  0,
         3,  3, 16,  3,  4,  3,  4,  0,  3,  0,  0, 10,  3,  8,  4,  9,  8,  3,
         3,  4,  6,  0,  9,  7,  9,  3,  9,  3,  4,  4,  0,  4,  0,  3, 11,  0,
         7,  3, 10,  6,  3,  1,  6, 12,  8, 10])
tensor([ 3,  9,  0,  6,  3,  1,  0,  0,  5,  0,  0,  0,  4,  6,  3,  4,  3,  0,
         6,  5,  0,  3,  9,  8,  3,  3,  0,  3,  0,  0,  4,  9,  6,  3,  0,  0,
         0,  3,  0,  3,  3,  3,  0,  0,  4,  9,  0,  0,  9,  3,  6, 10, 11,  1,
         3,  3,  9,  3,  6,  3,  0,  5,  0,  7])
tensor([ 3,  3,  0,  5,  9,  3,  0,  4,  3, 10,  0,  3,  3,  3,  4,  4,  4,  3,
         3,  0, 17, 12,  0,  6, 11,  3,  3, 12,  7,  7,  3,  0, 12,  3,  2,  0,
        10,  3,  1,  4,  0,  6,  3, 13,  3,  4,  3,  9,  5,  8,  8,  0,  0,  0,
         4,  4,  4,  0,  0,  0,  3,  3,  0,  0])
tensor([ 0,  0,  4,  3,  0,  3,  6, 13,  0,  0,  3,  3, 11,  3,  4,  5,  6,  4,
         0,  3,  3,  3,  0,  3, 15, 15,  7,  3,  3,  

tensor([ 6,  4,  4,  3, 12,  0,  4,  0, 12,  0,  0,  9,  8,  5,  4,  0,  3,  0,
         0, 13,  0,  3,  0,  3,  9,  0,  3,  6,  0,  9,  0,  3,  5,  6,  0, 15,
         0, 11, 11,  6,  7,  8,  8,  3,  0, 13,  1,  6,  0,  0,  0,  3,  3,  0,
         4,  3,  9,  0,  3,  6,  3,  3,  3,  4])
tensor([ 4,  3,  0,  3,  0,  3,  3,  3,  0,  0,  7,  8,  0,  3,  9,  4,  0,  3,
         0,  0,  4,  0,  2,  3, 17,  0,  0,  3,  3,  4,  3,  0,  3,  1, 13,  0,
         6,  4,  3,  3,  7,  2,  9,  6,  4,  0,  0,  3,  7,  3,  4,  0,  9,  3,
         0,  0,  6,  7,  1,  3,  0,  0,  0,  9])
tensor([ 4,  9,  3,  3,  0,  3,  3,  1,  0,  3,  4, 10,  6,  2,  3,  0,  6,  6,
        13,  9,  3, 10, 17,  3, 12,  0, 13,  3,  3,  0,  0,  6,  0, 16,  6,  9,
         3,  0,  7, 13,  3,  0,  1,  0,  6,  3,  9,  4,  0,  0,  3,  6,  5,  0,
         0,  0, 12,  2,  0, 10,  3,  6,  9,  3])
tensor([ 3,  4,  0,  3,  3,  0,  0,  0,  6,  0,  0,  5,  3,  3,  8,  2,  1,  6,
         0,  6,  3, 10, 17,  0,  4,  0,  6, 15,  3,  

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [25/25], Loss: 4.6246, Train acc: 0.9861, Test acc: 0.6377
Wall time: 58.5 s


In [39]:
class sur_model2(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size, n_tokens):
        super(sur_model2, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.embedding = OneHotEncoder(num_embeddings=n_tokens)
        self.seqnet = nn.Sequential(self.embedding,
            nn.Conv1d(input_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(4608, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
        out = self.seqnet(x)
        out = self.fc(out)
        return torch.nn.functional.log_softmax(out, dim=1)
class OneHotEncoder(nn.Module):
    """
    One hot encoded embeddings
    """
    def __init__(self, num_embeddings: int):
        super(OneHotEncoder, self).__init__()
        self.num_embeddings = num_embeddings
        self.embeddings_matrix = torch.eye(num_embeddings)
        
    def forward(self, token_indexes):
        return self.embeddings_matrix[token_indexes, :]

In [28]:
model1 = sur_model2(input_size, vocab_size, num_channels, output_size, n_tokens = len(vocab.idx_to_token))

In [29]:
layers = next(iter(model1.children()))

In [30]:
layers

OneHotEncoder()

In [31]:
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr = 0.001)

In [32]:
%%time
for_losses = []
n_total_steps = len(train_iter)


for epoch in range(25):
    total_loss, train_accuracy = train(model1, device, train_iter, optimizer, total = math.ceil(len(train_ds)/64))
    test_accuracy = test(model1, device, test_iter, total = math.ceil(len(test_ds)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [1/25], Loss: 167.6891, Train acc: 0.5139, Test acc: 0.5997


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

...

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [24/25], Loss: 11.3234, Train acc: 0.9630, Test acc: 0.6561


Training minibatch loop :   0%|          | 0/99 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/43 [00:00<?, ?it/s]

Epoch [25/25], Loss: 7.6282, Train acc: 0.9755, Test acc: 0.6705
CPU times: user 6min 48s, sys: 7.45 s, total: 6min 55s
Wall time: 6min 57s


## 3. Классификация обзоров на фильмы (ConvNet)

Датасет: https://disk.yandex.ru/d/tdinpb0nN_Dsrg

2.1 Создайте набор данных на основе файлов polarity/positive_reviews.csv (положительные отзывы) и polarity/negative_reviews.csv (отрицательные отзывы). Разбейте на обучающую и тестовую выборку.
  * токен = __слово__
  * данные для обучения в датасете представляются в виде последовательности индексов токенов
  * словарь создается на основе _только_ обучающей выборки. Для корректной обработки ситуаций, когда в тестовой выборке встретится токен, который не хранится в словаре, добавьте в словарь специальный токен `<UNK>`
  * добавьте предобработку текста

2.2. Обучите классификатор.
  
  * Для преобразования последовательности индексов в последовательность векторов используйте `nn.Embedding` 
    - подберите адекватную размерность вектора эмбеддинга: 
    - модуль `nn.Embedding` обучается

  * Используйте одномерные свертки и пулинг (`nn.Conv1d`, `nn.MaxPool1d`)
    - обратите внимание, что `nn.Conv1d` ожидает на вход трехмерный тензор размерности `(batch, embedding_dim, seq_len)`


2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)
* Целевое значение accuracy на валидации - 70+%

In [3]:
positive = pd.read_csv("negative_reviews.txt", sep="\n", header=None, names=["Review"])
nigative = pd.read_csv("positive_reviews.txt", sep="\n", header=None, names=["Review"])

In [4]:
positive["class"] = 0
nigative["class"] = 1
result = pd.concat([positive, nigative], ignore_index=True)

In [5]:
result

,Review,class
0,the rock is destined to be the 21st century's ...,0
1,"the gorgeously elaborate continuation of "" the...",0
2,effective but too-tepid biopic,0
3,if you sometimes like to go to the movies to h...,0
4,"emerges as something rare , an issue movie tha...",0
...,...,...
10657,a terrible movie that some people will neverth...,1
10658,there are many definitions of 'time waster' bu...,1
10659,"as it stands , crocodile hunter has the hurrie...",1
10660,the thing looks like a made-for-home-video qui...,1


In [6]:
X = result['Review']
y = result['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [7]:
words = []
for i in X:
    i = re.sub(r"[^a-zA-Z0-9\\]", " ", i)
    res = word_tokenize(i)
    self.max_seq_len = max(self.max_seq_len, len(res))
    words.append(res)
words = ['UNK']+[i[0] for i in words if i not in stopwords.words('english')]
uniqueWords = [] 
for i in words:
    if not i in uniqueWords:
        uniqueWords.append(i)
uniqueWords

NameError: name 'self' is not defined

In [ ]:
len(uniqueWords)

In [8]:
class Vocab:
      def __init__(self, data):
                words = []
                self.max_seq_len = 0
                for i in data:
                    k = i
                    kk = nltk.word_tokenize(k)
                    self.max_seq_len = max(self.max_seq_len, len(kk))
                    i = re.sub(r"[^a-zA-Z0-9\\]", " ", i)
                    res = word_tokenize(i)
                    words.append(res)
                words = ['UNK']+[i[0] for i in words if i not in stopwords.words('english')]
                uniqueWords = [] 
                for i in words:
                    if not i in uniqueWords:
                        uniqueWords.append(i)
                self.idx_to_token = {idx: val for idx, val in enumerate(uniqueWords)}
                self.token_to_idx = {val: idx for idx, val in self.idx_to_token.items()}
                self.vocab_len = len(self.idx_to_token)

In [9]:
vocab = Vocab(X)

In [10]:
vocab.idx_to_token

{0: 'UNK',
 1: 'the',
 2: 'effective',
 3: 'if',
 4: 'emerges',
 5: 'offers',
 6: 'perhaps',
 7: 'steers',
 8: 'take',
 9: 'this',
 10: 'what',
 11: 'wendigo',
 12: 'one',
 13: 'ultimately',
 14: 'an',
 15: 'illuminating',
 16: 'a',
 17: 'with',
 18: 'not',
 19: 'scores',
 20: 'occasionally',
 21: 'spiderman',
 22: 'at',
 23: 'it',
 24: 'guaranteed',
 25: 'light',
 26: 'while',
 27: 'though',
 28: 'cantet',
 29: 'ms',
 30: 'like',
 31: 'newton',
 32: 'fuller',
 33: 'compleja',
 34: 'karmen',
 35: 'gosling',
 36: 'tender',
 37: 'may',
 38: 'in',
 39: 'some',
 40: 'sandra',
 41: 'behind',
 42: 'everytime',
 43: 'manages',
 44: 'singer',
 45: 'you',
 46: 'whether',
 47: 'disney',
 48: 'just',
 49: 'morton',
 50: 'part',
 51: 'son',
 52: 'haneke',
 53: 'absorbing',
 54: 'painful',
 55: 'together',
 56: 'director',
 57: 'doug',
 58: 'katz',
 59: 'writer',
 60: 'i',
 61: 'chicago',
 62: 'steve',
 63: 'on',
 64: '4',
 65: 'edited',
 66: 'hip',
 67: 'between',
 68: 'somewhere',
 69: 'digital',

In [11]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
        abc = []
        review = nltk.word_tokenize(review)
        for i in review:
            if i in list(self.vocab.idx_to_token.values()):
                get_index = list(self.vocab.idx_to_token.values()).index(i)
                abc.append(get_index)
        while len(abc)<self.vocab.max_seq_len:
            abc.append(0)
        return torch.tensor(abc, dtype=torch.int64)


  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.vectorize(self.X.iloc[idx]), self.y.iloc[idx]

In [12]:
vocab = Vocab(X_train.tolist())
RD_train = ReviewDataset(X_train, y_train, vocab)
RD_test = ReviewDataset(X_test, y_test, vocab)

In [13]:
RD_train[15]

(tensor([ 13, 155,  42, 269, 269, 870,   3, 365, 629,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0]),
 0)

In [14]:
batch_size = 32
train_dl = DataLoader(RD_train, batch_size, shuffle=True)
test_dl = DataLoader(RD_test, batch_size, shuffle=False)

In [15]:
i = next(iter(train_dl))
i[0][1]

tensor([ 291,    5, 1596, 1612,    1,    5, 1404,  558,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])

In [16]:
# подсказал Тимлфей Лашуков и Саша Волненко
class Reshaper(nn.Module):

  def __init__(self):
    super().__init__()

  def forward(self, x):
    x = torch.permute(x, (0, 2, 1))
    return x

In [22]:
class sur_model2(nn.Module):
    def __init__(self, input_size, vocab_size, output_size, emb_size):
        super(sur_model2, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.seqnet = nn.Sequential(nn.Embedding(self.vocab_size, emb_size),
            Reshaper(),                       
            nn.Conv1d(emb_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(3072, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
          out = self.seqnet(x)
          out = self.fc(out)
          return out

In [23]:
input_size = vocab.max_seq_len
vocab_size = vocab.vocab_len
output_size = 2
emb_size = 64
model2 = sur_model2(input_size, vocab_size, output_size, emb_size)

In [24]:
criterion = nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model2.parameters(), lr = 0.001)

In [25]:
def train(model, device, train_dl, optimizer, total):
    #set model in train() mode:
    model.train()
    
    total_loss = 0.0
    total_samples = 0.0
    correct_samples = 0.0
    
    for i, (inputs, targets) in tqdm(enumerate(train_dl), total= total, desc='Training minibatch loop '):
        # Forward pass
        outputs = model(inputs)
        print(outputs)
        loss = criterion(outputs, targets)
        
        # Backward and optimize        
        # zero grad before new step        
        optimizer.zero_grad()                        
        loss.backward()
        optimizer.step()   

        # calculating the total_loss for checking
        total_loss += loss           
        
        # PREDICTIONS 
        total_samples += targets.shape[0]   
        _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
        correct_samples += torch.sum(predictions_indices==targets)

    train_accuracy = float(correct_samples) / total_samples        
    
    return total_loss, train_accuracy                        

In [26]:
# TESTING THE MODEL
def test(model, device, test_dl, total):
    #set model in eval() mode (it skips Dropout etc):
    model.eval()
    
    total_samples = 0.0
    correct_samples = 0.0 
    
    # set the requires_grad flag to false as we are in the test mode
    with torch.no_grad():
        for i, (inputs, targets) in tqdm(enumerate(test_dl), total=total, desc='Testing minibatch loop:'):                      
            # apply model to input data
            outputs = model(inputs)        
                       
            #PREDICTIONS
            total_samples += targets.shape[0]   
            _, predictions_indices = torch.max(outputs, 1) # dim=1 - dimension to reduce
            correct_samples += torch.sum(predictions_indices==targets)                    
        
    test_accuracy = correct_samples / total_samples        
    
    return test_accuracy              

In [29]:
## %%time
for_losses = []
n_total_steps = len(train_dl)


for epoch in range(25):
    total_loss, train_accuracy = train(model2, device, train_dl, optimizer, total = math.ceil(len(RD_train)/64))
    test_accuracy = test(model2, device, test_dl, total = math.ceil(len(RD_test)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/117 [00:00<?, ?it/s]

tensor([[ 0.0992,  0.0460],
        [ 0.0241,  0.0105],
        [-0.0048,  0.0873],
        [ 0.1542,  0.1310],
        [ 0.0142,  0.0712],
        [ 0.1275,  0.0091],
        [ 0.0461, -0.0992],
        [ 0.1218,  0.0414],
        [ 0.1064, -0.0141],
        [ 0.2008,  0.0701],
        [ 0.1059, -0.0041],
        [ 0.0781,  0.0911],
        [ 0.1302, -0.0042],
        [-0.0209,  0.0330],
        [ 0.0764, -0.0211],
        [ 0.1905,  0.0687],
        [ 0.1570, -0.0256],
        [ 0.1877, -0.0100],
        [ 0.0958,  0.0654],
        [ 0.2028,  0.0977],
        [ 0.2675, -0.0748],
        [ 0.1085,  0.0348],
        [ 0.1137,  0.1016],
        [ 0.1982,  0.1289],
        [ 0.0878, -0.0496],
        [ 0.0689,  0.0730],
        [ 0.0154,  0.1029],
        [ 0.0727,  0.0049],
        [ 0.1828, -0.0064],
        [ 0.1162, -0.0775],
        [ 0.0260,  0.0641],
        [ 0.0229,  0.0859]], grad_fn=<AddmmBackward0>)
tensor([[-0.2387,  0.4343],
        [ 0.9481, -0.8467],
        [ 0.4349, -0.

tensor([[0.0657, 0.5374],
        [0.1273, 0.4848],
        [0.2096, 0.2431],
        [0.2851, 0.3549],
        [0.0542, 0.5399],
        [0.2460, 0.2678],
        [0.1102, 0.5286],
        [0.2662, 0.2813],
        [0.2455, 0.3071],
        [0.0936, 0.4968],
        [0.2219, 0.3756],
        [0.1282, 0.5090],
        [0.0514, 0.5934],
        [0.0461, 0.5715],
        [0.1074, 0.4690],
        [0.0975, 0.5032],
        [0.2331, 0.3002],
        [0.0193, 0.6074],
        [0.1446, 0.5041],
        [0.1748, 0.4516],
        [0.2523, 0.4426],
        [0.0677, 0.4695],
        [0.1963, 0.5034],
        [0.2719, 0.3295],
        [0.2154, 0.2238],
        [0.2534, 0.3527],
        [0.2610, 0.4201],
        [0.2367, 0.2387],
        [0.2045, 0.4409],
        [0.2513, 0.2530],
        [0.1135, 0.5271],
        [0.2465, 0.2621]], grad_fn=<AddmmBackward0>)
tensor([[ 0.2114,  0.3829],
        [ 0.2784,  0.2400],
        [ 0.2451,  0.3586],
        [ 0.2279,  0.3844],
        [ 0.2315,  0.1723],
 

tensor([[0.0900, 0.6021],
        [0.1149, 0.5910],
        [0.1147, 0.4852],
        [0.1434, 0.4621],
        [0.1296, 0.5687],
        [0.1404, 0.4834],
        [0.1229, 0.5075],
        [0.1484, 0.5468],
        [0.0823, 0.5686],
        [0.1054, 0.5917],
        [0.1517, 0.5276],
        [0.1273, 0.4153],
        [0.1344, 0.4284],
        [0.1114, 0.6119],
        [0.1459, 0.4670],
        [0.1490, 0.4870],
        [0.0947, 0.4929],
        [0.1421, 0.4706],
        [0.1441, 0.4778],
        [0.0842, 0.5451],
        [0.0978, 0.6075],
        [0.1053, 0.5805],
        [0.1003, 0.5019],
        [0.1345, 0.4328],
        [0.1247, 0.6126],
        [0.1322, 0.6200],
        [0.1556, 0.5749],
        [0.1386, 0.5742],
        [0.1429, 0.4504],
        [0.1282, 0.6174],
        [0.1470, 0.6171],
        [0.1446, 0.4478]], grad_fn=<AddmmBackward0>)
tensor([[0.2083, 0.4960],
        [0.1845, 0.5523],
        [0.1848, 0.5402],
        [0.1804, 0.4975],
        [0.1897, 0.4921],
        [0.

tensor([[0.1893, 0.3481],
        [0.2096, 0.2495],
        [0.2511, 0.2252],
        [0.1827, 0.3398],
        [0.2225, 0.2423],
        [0.2324, 0.2478],
        [0.1806, 0.3488],
        [0.1777, 0.3326],
        [0.2449, 0.1973],
        [0.1794, 0.3221],
        [0.1795, 0.3698],
        [0.1868, 0.3632],
        [0.1777, 0.3490],
        [0.2052, 0.2503],
        [0.2175, 0.3717],
        [0.2504, 0.2182],
        [0.2416, 0.2175],
        [0.1856, 0.3601],
        [0.2211, 0.2486],
        [0.1567, 0.3609],
        [0.2061, 0.2305],
        [0.2099, 0.2624],
        [0.1822, 0.2688],
        [0.2159, 0.1717],
        [0.2387, 0.2412],
        [0.1893, 0.3693],
        [0.1774, 0.3558],
        [0.1935, 0.3536],
        [0.2266, 0.2278],
        [0.2320, 0.2554],
        [0.1805, 0.2955],
        [0.1924, 0.3437]], grad_fn=<AddmmBackward0>)
tensor([[0.1176, 0.3072],
        [0.1580, 0.3766],
        [0.1512, 0.2883],
        [0.1360, 0.3277],
        [0.1512, 0.2866],
        [0.

...

tensor([[ 0.7650, -0.1647],
        [ 0.6452, -0.1173],
        [ 0.4905, -0.0110],
        [ 0.2577,  0.0784],
        [ 0.6483, -0.0803],
        [-0.7755,  0.6913],
        [ 0.4106,  0.0494],
        [ 0.3038,  0.1002],
        [ 0.2248,  0.0145],
        [ 0.0926,  0.1709],
        [ 0.0700,  0.5724],
        [ 0.0966,  0.1238],
        [-0.6206,  0.6921],
        [ 1.5839, -0.4449],
        [ 0.7019, -0.1345],
        [ 1.4242, -0.3318],
        [ 1.1778, -0.2221],
        [-0.2066,  0.4745],
        [ 0.4521, -0.1001],
        [ 0.4601, -0.0083],
        [-0.0532,  0.5802],
        [-0.2360,  0.3498],
        [ 0.4854, -0.1338],
        [ 0.1046,  0.1342],
        [-0.5603,  0.5779],
        [-0.5802,  0.5300],
        [ 0.4854, -0.0538],
        [-0.7206,  0.7666],
        [-0.2262,  0.5643],
        [ 0.5725, -0.0372],
        [ 0.7000, -0.2711],
        [ 2.1462, -0.6215]], grad_fn=<AddmmBackward0>)
tensor([[-0.2768,  0.8170],
        [ 0.3983,  0.0322],
        [ 0.5775, -0.

tensor([[-0.6043,  0.4056],
        [ 1.1757, -0.4435],
        [-0.4645,  0.5480],
        [-0.4052,  0.2985],
        [ 0.2023, -0.0558],
        [-0.1493,  0.2701],
        [-0.7540,  0.4464],
        [ 0.0226,  0.2787],
        [ 1.2653, -0.4860],
        [ 0.0392,  0.1343],
        [-0.0071,  0.1312],
        [ 0.1797,  0.0492],
        [-0.1937,  0.2007],
        [-0.3490,  0.2643],
        [ 0.4883, -0.1560],
        [ 0.8660, -0.3305],
        [-0.5000,  0.5825],
        [ 0.1843,  0.0749],
        [-0.2920,  0.8413],
        [ 0.7523, -0.2143],
        [-0.1543,  0.3731],
        [-0.6102,  0.5276],
        [ 0.6863, -0.4731],
        [ 0.2742,  0.0103],
        [ 0.8922, -0.3412],
        [-0.5719,  0.9073],
        [-0.2468,  0.5210],
        [ 0.8000, -0.3859],
        [ 1.2079, -0.4589],
        [ 1.3962, -0.7417],
        [-0.1313,  0.3041],
        [-0.5556,  0.4822]], grad_fn=<AddmmBackward0>)
tensor([[ 0.0841,  0.1070],
        [ 1.3694, -0.5492],
        [ 0.6353, -0.

tensor([[-0.1138,  0.3301],
        [ 1.4457, -1.1367],
        [-0.0853,  0.2638],
        [ 0.4323, -0.2097],
        [ 0.5793, -0.4177],
        [ 0.3649, -0.1332],
        [ 0.2218, -0.0153],
        [-0.1363,  0.2281],
        [-0.1587,  0.5188],
        [-0.0230,  0.1876],
        [ 0.0172,  0.0704],
        [ 0.1228,  0.0885],
        [-0.2923,  0.3876],
        [ 0.6256, -0.3833],
        [ 0.6836, -0.5946],
        [-0.0209,  0.0941],
        [-0.1807,  0.3322],
        [ 0.5388, -0.5633],
        [-0.0023,  0.2210],
        [ 0.0537,  0.1249],
        [-0.2477,  0.2915],
        [-0.0870,  0.3813],
        [ 0.2382,  0.1307],
        [ 0.0119,  0.1391],
        [-0.2660,  0.1918],
        [ 0.9661, -0.8104],
        [-0.2535,  0.3006],
        [ 0.7985, -0.4980],
        [ 0.7618, -0.5424],
        [-0.0055,  0.0941],
        [-0.1943,  0.5163],
        [-0.1393,  0.4035]], grad_fn=<AddmmBackward0>)
tensor([[ 1.6941, -1.4380],
        [-0.3299,  0.5573],
        [ 0.7421, -0.

tensor([[-0.1806,  0.3857],
        [-0.1256,  0.2553],
        [-0.0075,  0.1883],
        [ 0.5729, -0.4861],
        [-0.4368,  0.3357],
        [-0.3398,  0.5835],
        [-0.5761,  0.4754],
        [ 0.3918, -0.0874],
        [ 0.0505,  0.1587],
        [-0.2083,  0.3200],
        [ 0.3969, -0.1052],
        [-0.1812,  0.4874],
        [-0.7667,  0.3738],
        [-0.1051,  0.3406],
        [ 0.3013, -0.2005],
        [ 0.2572, -0.1758],
        [ 0.2911,  0.0268],
        [-0.2017,  0.4390],
        [ 0.0648,  0.1252],
        [-0.1664,  0.2996],
        [ 0.6894, -0.2441],
        [-0.1142,  0.1650],
        [-0.5057,  0.3518],
        [ 0.7754, -0.3356],
        [ 0.3717, -0.0648],
        [ 0.0396,  0.1339],
        [-0.0171,  0.1354],
        [ 0.0688,  0.1368],
        [-0.2018,  0.4225],
        [ 0.8359, -0.3132],
        [ 0.3158, -0.0552],
        [ 0.9764, -0.5369]], grad_fn=<AddmmBackward0>)
tensor([[-0.1315,  0.3621],
        [ 0.6697, -0.4958],
        [-0.0641,  0.

KeyboardInterrupt: 

In [101]:
class sur_model2(nn.Module):
    def __init__(self, input_size, vocab_size, num_channels, output_size, n_tokens):
        super(sur_model2, self).__init__()

        self.input_size = input_size
        self.output_size = output_size
        self.vocab_size = vocab_size
        self.embedding = OneHotEncoder(num_embeddings=n_tokens)
        self.seqnet = nn.Sequential(self.embedding,
            nn.Conv1d(input_size, 64, 3),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(64, 256, 2),
            nn.BatchNorm1d(256),
            nn.ELU(),
            nn.MaxPool1d(2),                        
            nn.Conv1d(256, 512, 2),
            nn.BatchNorm1d(512),
            nn.ELU(),
            nn.MaxPool1d(2),
            nn.Flatten(), 
            nn.Linear(111616, 256),
            nn.ELU(),
            nn.Linear(256, 128),
            nn.ELU(),
            nn.Linear(128, 64)
        )
        self.fc = nn.Linear(64, output_size)


    def forward(self, x):
        out = self.seqnet(x)
        out = self.fc(out)
        return torch.nn.functional.log_softmax(out, dim=1)
class OneHotEncoder(nn.Module):
    """
    One hot encoded embeddings
    """
    def __init__(self, num_embeddings: int):
        super(OneHotEncoder, self).__init__()
        self.num_embeddings = num_embeddings
        self.embeddings_matrix = torch.eye(num_embeddings)
        
    def forward(self, token_indexes):
        return self.embeddings_matrix[token_indexes, :]

In [102]:
len(vocab.idx_to_token)

1759

In [103]:
input_size = vocab.max_seq_len
vocab_size = vocab.vocab_len
output_size = 2
num_channels = 300
model3 = sur_model2(input_size, vocab_size, num_channels, output_size, n_tokens = len(vocab.idx_to_token))

In [104]:
model3

sur_model2(
  (embedding): OneHotEncoder()
  (seqnet): Sequential(
    (0): OneHotEncoder()
    (1): Conv1d(62, 64, kernel_size=(3,), stride=(1,))
    (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ELU(alpha=1.0)
    (4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv1d(64, 256, kernel_size=(2,), stride=(1,))
    (6): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ELU(alpha=1.0)
    (8): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv1d(256, 512, kernel_size=(2,), stride=(1,))
    (10): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ELU(alpha=1.0)
    (12): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=111616, out_features=256, bias=True)
    (15): ELU(alpha=1.0)
    (16): 

In [105]:
criterion = nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model3.parameters(), lr = 0.001)

In [106]:
## %%time
for_losses = []
n_total_steps = len(train_dl)


for epoch in range(25):
    total_loss, train_accuracy = train(model3, device, train_dl, optimizer, total = math.ceil(len(RD_train)/64))
    test_accuracy = test(model3, device, test_dl, total = math.ceil(len(RD_test)/64))
    for_losses.append(total_loss.cpu().detach().numpy())
    print (f'Epoch [{epoch+1}/{25}], Loss: {total_loss:.4f}, Train acc: {train_accuracy:.4f}, Test acc: {test_accuracy:.4f}')                                    

Training minibatch loop :   0%|          | 0/117 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [1/25], Loss: 163.1854, Train acc: 0.5943, Test acc: 0.6046


Training minibatch loop :   0%|          | 0/117 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/50 [00:00<?, ?it/s]

...

Training minibatch loop :   0%|          | 0/117 [00:00<?, ?it/s]

Testing minibatch loop::   0%|          | 0/50 [00:00<?, ?it/s]

Epoch [17/25], Loss: 7.3844, Train acc: 0.9871, Test acc: 0.5449


Training minibatch loop :   0%|          | 0/117 [00:00<?, ?it/s]

KeyboardInterrupt: 